# GABBR1
Pathogenic vartiants in GABBR1 are associated with [Neurodevelopmental disorder with language delay and variable cognitive abnormalities](https://omim.org/entry/620502) (NEDLC). 

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.56


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/GABBR1_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Sleep abnormality,Optic nerve hypoplasia,Nystagmus,Scoliosis,Increased femoral anteversion,Dry skin,Nail dystrophy,Long philtrum,Wide nasal bridge,Clinodactyly of the 5th finger
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0002360,HP:0000609,HP:0000639,HP:0002650,HP:0012427,HP:0000958,HP:0008404,HP:0000343,HP:0000431,HP:0004209
1,PMID:36103875,GABBR1 monoallelic de novo variants linked to neurodevelopmental delay and epilepsy,Individual 1,NaN,OMIM:620502,Neurodevelopmental disorder with language delay and variable cognitive abnormalities,NM_001470.4,c.1603G>A,na,p.Ala535Thr,...,excluded,excluded,excluded,excluded,excluded,excluded,excluded,excluded,excluded,excluded


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 33 fields


In [5]:
GABBR1_transcript = "NM_001470.4"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="GABBR1",
                          transcript=GABBR1_transcript,
                          allele_1_column_name="allele_1")

In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,4,"c.1104G>C, c.1603G>A , c.2018G>A, c.1190C>T"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [8]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Individual 1 (FEMALE; P2Y6M),Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502),NM_001470.4:c.1603G>A (heterozygous),Motor delay (HP:0001270); Delayed speech and language development (HP:0000750); Hypotonia (HP:0001252); Abnormal repetitive mannerisms (HP:0000733); excluded: Seizure (HP:0001250); excluded: Autistic behavior (HP:0000729); excluded: Attention deficit hyperactivity disorder (HP:0007018); excluded: Encopresis (HP:0040183); excluded: Sleep abnormality (HP:0002360); excluded: Optic nerve hypoplasia (HP:0000609); excluded: Nystagmus (HP:0000639); excluded: Scoliosis (HP:0002650); excluded: Increased femoral anteversion (HP:0012427); excluded: Dry skin (HP:0000958); excluded: Nail dystrophy (HP:0008404); excluded: Long philtrum (HP:0000343); excluded: Wide nasal bridge (HP:0000431); excluded: Clinodactyly of the 5th finger (HP:0004209)
Individual 2 (FEMALE; P4Y6M),Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502),NM_001470.4:c.1104G>C (heterozygous),Motor delay (HP:0001270); Delayed speech and language development (HP:0000750); Hypotonia (HP:0001252); Seizure (HP:0001250); Intellectual disability (HP:0001249); Sleep abnormality (HP:0002360); Optic nerve hypoplasia (HP:0000609); Nystagmus (HP:0000639); Scoliosis (HP:0002650); Long philtrum (HP:0000343); Wide nasal bridge (HP:0000431); Clinodactyly of the 5th finger (HP:0004209); excluded: Abnormal repetitive mannerisms (HP:0000733); excluded: Autistic behavior (HP:0000729); excluded: Attention deficit hyperactivity disorder (HP:0007018); excluded: Encopresis (HP:0040183); excluded: Increased femoral anteversion (HP:0012427); excluded: Dry skin (HP:0000958); excluded: Nail dystrophy (HP:0008404)
Individual 3 (MALE; P13Y),Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502),NM_001470.4:c.1190C>T (heterozygous),Delayed speech and language development (HP:0000750); Abnormal repetitive mannerisms (HP:0000733); Intellectual disability (HP:0001249); Attention deficit hyperactivity disorder (HP:0007018); Encopresis (HP:0040183); Sleep abnormality (HP:0002360); excluded: Motor delay (HP:0001270); excluded: Hypotonia (HP:0001252); excluded: Seizure (HP:0001250); excluded: Autistic behavior (HP:0000729); excluded: Optic nerve hypoplasia (HP:0000609); excluded: Nystagmus (HP:0000639); excluded: Scoliosis (HP:0002650); excluded: Increased femoral anteversion (HP:0012427); excluded: Dry skin (HP:0000958); excluded: Nail dystrophy (HP:0008404); excluded: Long philtrum (HP:0000343); excluded: Wide nasal bridge (HP:0000431); excluded: Clinodactyly of the 5th finger (HP:0004209)
Individual 4 (FEMALE; P17Y),Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502),NM_001470.4:c.2018G>A (heterozygous),Motor delay (HP:0001270); Delayed speech and language development (HP:0000750); Autistic behavior (HP:0000729); Attention deficit hyperactivity disorder (HP:0007018); Increased femoral anteversion (HP:0012427); Dry skin (HP:0000958); Nail dystrophy (HP:0008404); excluded: Hypotonia (HP:0001252); excluded: Abnormal repetitive mannerisms (HP:0000733); excluded: Seizure (HP:0001250); excluded: Intellectual disability (HP:0001249); excluded: Encopresis (HP:0040183); excluded: Sleep abnormality (HP:0002360); excluded: Optic nerve hypoplasia (HP:0000609); excluded: Nystagmus (HP:0000639); excluded: Scoliosis (HP:0002650); excluded: Long philtrum (HP:0000343); excluded: Wide nasal bridge (HP:0000431); excluded: Clinodactyly of the 5th finger (HP:0004209)


In [9]:
encoder.output_individuals_as_phenopackets(individuals)

We output 4 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [10]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
PMID = "PMID:36103875"
hpo_table_creator = builder.autosomal_recessive(PMID).build()

[pyphetools] Ingested 4 GA4GH phenopackets.
We found a total of 19 unique HPO terms
Extracted disease: Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502)


In [11]:
df = hpo_table_creator.get_dataframe()
df.head(5)

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620502,Neurodevelopmental disorder with language delay and variable cognitive abnormalities,HP:0001270,Motor delay,,,3/4,,,,,PMID:36103875,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620502,Neurodevelopmental disorder with language delay and variable cognitive abnormalities,HP:0000750,Delayed speech and language development,,,4/4,,,,,PMID:36103875,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620502,Neurodevelopmental disorder with language delay and variable cognitive abnormalities,HP:0001252,Hypotonia,,,2/4,,,,,PMID:36103875,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620502,Neurodevelopmental disorder with language delay and variable cognitive abnormalities,HP:0000733,Abnormal repetitive mannerisms,,,2/4,,,,,PMID:36103875,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620502,Neurodevelopmental disorder with language delay and variable cognitive abnormalities,HP:0001250,Seizure,,,1/4,,,,,PMID:36103875,PCS,ORCID:0000-0002-0736-9199


In [12]:
hpo_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620502.tab
